<a href="https://www.kaggle.com/code/sahilr05/movie-recommendation?scriptVersionId=106212034" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast
from sklearn.feature_extraction.text import CountVectorizer
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [2]:
movies = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv")
credit = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv")

In [3]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [4]:
credit.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [5]:
# Not suggest to best movies
# revenue,budget, homepage, original_language, production_companies, release_date, runtime, spoken_languages, status,vote_count,vote_average

In [6]:
# we create movie tage based on that we predict next 5 movies
# so only category feature tack 
# id is important for find movies data
movies = movies[["id","title","overview","tagline","genres","keywords",]]

In [7]:
# merge movies & credit data set based on title
movies = movies.merge(credit,on="title")

In [8]:
movies.drop(["movie_id"],axis=1,inplace=True)

Feature Engineering

In [9]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

def convert2(text):
    a = []
    count = 0
    for i in ast.literal_eval(text):
        if count < 3:
            a.append(i['name']) 
        count +=1
    return a

def convert3(data):
    a = []
    for i in ast.literal_eval(data):
        if i['job'] == 'Director':
            a.append(i['name'])
    return a

In [10]:
movies["genres"] = movies["genres"].apply(convert)
movies["keywords"] = movies["keywords"].apply(convert)
movies["cast"] = movies["cast"].apply(convert2)
movies["crew"] = movies["crew"].apply(convert3)

In [11]:
movies.dropna(inplace=True)
movies.isnull().sum()

id          0
title       0
overview    0
tagline     0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [12]:
def remove_space(data):
    a= []
    for i in data:
        a.append(i.replace(" ",""))
    return a

In [13]:
movies["crew"] = movies["crew"].apply(remove_space)
movies["cast"] = movies["cast"].apply(remove_space)
movies["keywords"] = movies["keywords"].apply(remove_space)
movies["genres"] = movies["genres"].apply(remove_space)

In [14]:
movies["overview"] = movies["overview"].apply(lambda x : x.split())
movies["tagline"] = movies["tagline"].apply(lambda x : x.split())

In [15]:
movies["tag"] = movies["overview"] + movies["tagline"] + movies["genres"] + movies["keywords"] + movies["cast"]+ movies["crew"]

In [16]:
movies["tag"] = movies["tag"].apply(lambda x: " ".join(x))

In [17]:
movies

,id,title,overview,tagline,genres,keywords,cast,crew,tag
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Enter, the, World, of, Pandora.]","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[At, the, end, of, the, world,, the, adventure...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[A, Plan, No, One, Escapes]","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[The, Legend, Ends]","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],Following the death of District Attorney Harve...
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Lost, in, our, world,, found, in, another.]","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...,...,...
4801,124606,Bang,"[A, young, woman, in, L.A., is, having, a, bad...","[Sometimes, you've, got, to, break, the, rules]",[Drama],"[gang, audition, policefake, homeless, actress]","[DarlingNarita, PeterGreene, MichaelNewland]",[AshBaron-Cohen],A young woman in L.A. is having a bad day: she...
4802,14337,Primer,"[Friends/fledgling, entrepreneurs, invent, a, ...","[What, happens, if, it, actually, works?]","[ScienceFiction, Drama, Thriller]","[distrust, garage, identitycrisis, timetravel,...","[ShaneCarruth, DavidSullivan, CaseyGooden]",[ShaneCarruth],Friends/fledgling entrepreneurs invent a devic...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui...","[He, didn't, come, looking, for, trouble,, but...","[Action, Crime, Thriller]","[unitedstates–mexicobarrier, legs, arms, paper...","[CarlosGallardo, JaimedeHoyos, PeterMarquardt]",[RobertRodriguez],El Mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended...","[A, newlywed, couple's, honeymoon, is, upended...","[Comedy, Romance]",[],"[EdwardBurns, KerryBishé, MarshaDietlein]",[EdwardBurns],A newlywed couple's honeymoon is upended by th...


In [18]:
movies.drop(["overview","tagline","genres","keywords","cast","crew"],axis=1,inplace=True)

In [19]:
cv  = CountVectorizer(max_features=6000,stop_words="english")

In [20]:
vector =cv.fit_transform(movies["tag"]).toarray()

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
cosim= cosine_similarity(vector)

In [22]:
def recommend(name):
    # index fetch
    index = np.where(movies["title"]==name)[0][0]
    similar_items = sorted(list(enumerate(cosim[index])),key=lambda x:x[1],reverse=True)[0:5]
    
    data = []
    for i in similar_items:
        v = movies[movies.index== i[0]]["title"]
        v = list(v)
        data.append(v)
    
    return pd.DataFrame(data)

In [23]:
recommend("Avatar")

,0
0,Avatar
1,Shark Tale
2,Runaway Bride
3,X-Men: Days of Future Past
4,The Man
